# Monte Carlo Markov Chain

Code adapted from [MCMC Intuition for Everyone](https://towardsdatascience.com/mcmc-intuition-for-everyone-5ae79fff22b1) by Rahul Agarwal.

In [2]:
import random
import numpy as np
import pylab as pl
import scipy.special as ss
%matplotlib inline

Q = np.matrix([[0.9  , 0.075, 0.025],
               [0.15 , 0.80 , 0.05 ],
               [0.25 , 0.25 , 0.5  ]])

In [3]:
# state 1
init_s = np.matrix([[1, 0, 0]])
np.dot(init_s, Q)

matrix([[0.9  , 0.075, 0.025]])

In [4]:
def find_stationary(init_s, Q, tol=10e-9):
    """
    Find a stationary state in a Markov Chain.
    
    Args
    ----
        init_s : initial state
        Q : matrix of transition probabilities
        tol : a tolerance limit
    """
    
    epsilon = 1
    while epsilon > tol:
        next_s = np.dot(init_s, Q)
        epsilon = np.sqrt(np.sum(np.square(next_s - init_s)))
        init_s = next_s
    
    # test equality
    try:
        assert init_s.all() == next_s.all()
    except AssertionError:
        print('stationary state not found.')
        
    return next_s

In [5]:
stat_s1 = find_stationary(init_s, Q)
stat_s1

matrix([[0.62500002, 0.31249998, 0.0625    ]])

In [6]:
# state 2
init_s = np.matrix([[0, 1, 0]])
stat_s2 = find_stationary(init_s, Q)
stat_s2

matrix([[0.62499998, 0.31250002, 0.0625    ]])

In [7]:
# state 3
init_s = np.matrix([[0, 0, 1]])
stat_s3 = find_stationary(init_s, Q)
stat_s3

matrix([[0.62499998, 0.31250002, 0.0625    ]])

> "The stationary state distribution is important because it lets you define the probability for every state of a system at a random time."

In [8]:
# Lets define our Beta Function to generate s for any particular state. We don't care for the normalizing constant here.
def beta_s(w,a,b):
    return w**(a-1)*(1-w)**(b-1)

# This Function returns True if the coin with probability P of heads comes heads when flipped.
def random_coin(p):
    unif = random.uniform(0,1)
    if unif>=p:
        return False
    else:
        return True

# This Function runs the MCMC chain for Beta Distribution.
def beta_mcmc(N_hops,a,b):
    states = []
    cur = random.uniform(0,1)
    for i in range(0,N_hops):
        states.append(cur)
        next = random.uniform(0,1)
        ap = min(beta_s(next,a,b)/beta_s(cur,a,b),1) # Calculate the acceptance probability
        if random_coin(ap):
            cur = next
    return states[-1000:] # Returns the last 100 states of the chain

In [13]:
# Actual Beta PDF == has division by zero errors
def beta(a, b, i):
    e1 = ss.gamma(a + b)
    e2 = ss.gamma(a)
    e3 = ss.gamma(b)
    e4 = i ** (a - 1)
    e5 = (1 - i) ** (b - 1)
    return (e1/(e2*e3)) * e4 * e5

In [17]:
result = beta(0.1, 0.1, 100)

In [19]:
type(result)

numpy.complex128

In [20]:
result

(-1.2227729158498092e-05-3.973030043797453e-06j)

In [14]:
#pl.rcParams['figure.figsize'] = (17.0, 4.0)
#plot_beta(0.1, 0.1)
#plot_beta(1, 1)
#plot_beta(2, 3)

In [55]:
def generate_beta_state(weight, alpha, beta):
    """
    Generate 's' for a particular state, given a
    alpha and beta parameters and a weight.
    
    NB - we don't care about the normalizing constanct 'C'
    """
    term1 = weight**(alpha - 1)
    term2 = (1 - weight)**(beta - 1)
    return term1 * term2

def flip_coin(prob_tresh):
    """
    Flip a random coin from a uniform distribution,
    given a probability threshold for "heads".
    """
    unif = random.uniform(0, 1)
    if unif >= prob_tresh:
        return False
    else:
        return True

def run_beta_chain(n_hops, alpha, beta, n_states=1000):
    """
    Run an MCMC chain for the beta distribution, 
    given a number of hops, alpha and beta, and the 
    desired last number of states.
    """
    states = []
    current_state = random.uniform(0, 1)
    for i in range(0, n_hops):
        states.append(current_state)
        next_state = random.uniform(0, 1)
        numer = generate_beta_state(next_state, alpha, beta)
        denom = generate_beta_state(current_state, alpha, beta)
        acceptance_prob = min(numer/denom, 1)
        if flip_coin(acceptance_prob):
            current = next_state
    return states[-n_states:]

In [56]:
flip_coin(.5)

False

In [1]:
#run_beta_chain(100000, 0.1, 0.4, n_states=10) # Eh?

In [54]:
#beta_mcmc(1000, 0.1, 0.1)